In [34]:
import random, os
import cv2
import numpy as np
from imutils import contours
from skimage.filters import threshold_local

In [35]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
 
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
 
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
 
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
 
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    # return the warped image
    return warped

In [36]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
 
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
 
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
 
    # return the ordered coordinates
    return rect

In [210]:
def resize_image(image):
    r = 500.0 / image.shape[1]
    dim = (500, int(image.shape[0] * r))
    kernel = np.ones((5,5),np.uint8)

    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized = cv2.bilateralFilter(resized, 9 , 50, 50)
    resized = cv2.GaussianBlur(resized,(3,3),0)
    return resized

In [211]:
def morph_canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5,5),np.uint16)

    gray = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    gray = cv2.dilate(gray,kernel,iterations = 5)
    edged = cv2.Canny(gray, 50, 200)

    cv2.imshow("Edged", edged)
    cv2.waitKey(0)
    return edged

In [212]:
def contour_warp_corp(image1,image2):
    cnts = cv2.findContours(image1.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
    screenCnt = 0
 
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break

    cv2.drawContours(image2, [screenCnt], -1, (0, 255, 0), 2)
    cv2.imshow("Outline", image2)
    cv2.waitKey(0)
    
    warped = four_point_transform(image2, screenCnt.reshape(4,2))
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = threshold_local(warped, 11, offset = 10, method = 'gaussian')
    warped = (warped > T).astype("uint8") * 255

    cv2.imshow('Cropped',warped)
    cv2.waitKey(0)
    return warped

In [213]:
path = r'C:\Users\Lap Mania\DealSmash\Project 1 (Receipts)'

In [275]:
random_filename1 = random.choice([
    x for x in os.listdir(path)

         if os.path.isfile(os.path.join(path, x))
])

In [276]:
print(random_filename1)

91_receipt_94962.png


In [277]:
img_input = cv2.imread(random_filename1)

In [278]:
x = resize_image(img_input)

In [279]:
y = morph_canny(x)

In [280]:
z = contour_warp_corp(y,x)

error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\drawing.cpp:2506: error: (-215) npoints > 0 in function cv::drawContours
